In [273]:
from bs4 import BeautifulSoup as bs
import requests
import re
import spacy

In [274]:
headers = {"User-Agent": "teng030918@gmail.com"}
def have_uci(str):
    if "University of California, Irvine" in str or "University of California Irvine" in str or "UC Irvine" in str or "UC, Irvine" in str:
        return True
    return False
url = "https://www.sec.gov/Archives/edgar/data/1531177/000114036124022158/ny20024323x1_def14a.htm"

def get_name_from_url(url, headers):
    response = requests.get(url, headers=headers)
    doc = bs(response.content, 'html.parser')
    doc_text = doc.get_text()
    # print(len(doc_text))
    
    list_of_s = []
    list_of_p = []
    modified_p = []
    con_string = ""
    paragraphs = re.split('\n', doc_text)
    flag = 0
    for i, paragraph in enumerate(paragraphs):
        if not paragraph.endswith("."):
            if flag == 0:
                con_string += paragraph.strip()
                flag = 1
            else:
                con_string += paragraph.strip()  
        else:
            if flag == 1:
                con_string += paragraph.strip()
                modified_p.append(con_string)
                con_string = ""
                flag = 0
            else:
                modified_p.append(paragraph.strip())
    if con_string != "":
        modified_p.append(con_string)
    # print(len(modified_p))
    
    
    for p in modified_p:
        if have_uci(p):
                s = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', p)
                list_of_p.append(p)
                for i in s:
                    if have_uci(i):
                        list_of_s.append(i)
    # print(len(list_of_p))
    
    nlp = spacy.load("en_core_web_sm")
    name_set = {}
    
    
    def find_name(paragraph):
    
        doc = nlp(paragraph)
        flag = 0
        the_name = ""
        line = ""
        # Extract names of people related to the University of California, Irvine
        for ent in reversed(doc.ents):
            if ent.label_ == "PERSON":
                if flag == 1:
                    return ent.text, line
                            
            elif have_uci(ent.sent.text):
                flag = 1
                line = ent.sent.text
        return the_name, line
        
    for p in list_of_p:
        result, line = find_name(p)
        if len(result) > 0:
            name_set[result.strip()] = line
            
    return name_set
    

In [280]:
import csv
url_list = {}
with open("sec_outputs.csv", 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if row[6] == "DEF 14A":
            url_list[row[2]] = row[0]

In [281]:
print(len(url_list))

224


In [282]:
test_url = "https://www.sec.gov/Archives/edgar/data/1852575/000119312523095400/d494730ddef14a.htm"
test_result = get_name_from_url(test_url, headers)
print(test_result)

{'Counsel': 'Mr.\xa0Look holds a B.S. in Computer Science from the University of California, Irvineand a J.D. from the Pepperdine University School of Law.', 'Muzzy': 'Mr.\xa0Muzzy graduated with a B.A. in Social Ecology from the University of California, Irvine, and has an MBAfrom Pepperdine University.  '}


In [286]:
with open("partial_names.csv", 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["last_name", "url", "company cik","sentence"])

In [287]:
with open("partial_names.csv", 'a') as csvfile:
    writer = csv.writer(csvfile)
    # counter = 0
    for url in url_list.keys():
        # if counter >= 10:
        #     break
        pairs = get_name_from_url(url, headers)
        if len(pairs) > 0:
            for name in pairs.keys():
                writer.writerow([name, url, url_list[url], pairs[name]])
        else:
            writer.writerow(["", url, url_list[url], ""])
        # counter += 1

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
